In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [0]:
data=pd.read_csv('gdrive/My Drive/Colab Notebooks/Predictive Modeling/Fraud/DATA/3. fraud_remove_nulls_fill_random_distribution/test-fraud_remove_nulls_fill_random_distribution.csv',compression='gzip')#2

In [0]:
cat_cols=['ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
       'addr1', 'addr2', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9',
       'id_12', 'id_13', 'id_15', 'id_16', 'id_17', 'id_19', 'id_20', 'id_28',
       'id_29', 'id_31', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType',
       '_Weekdays', '_Hours', '_Days', 'P_emaildomain_bin',
       'P_emaildomain_suffix', 'R_emaildomain_bin', 'R_emaildomain_suffix',
       'device_name', 'had_id', '_Month']

In [6]:
for i in range(len(cat_cols)):
  col=cat_cols[i]
  d[col]=data[col].astype(str) 

NameError: ignored

In [26]:
to_dummy=d[cat_cols].nunique()[data[cat_cols].nunique()<10].index
len(to_dummy)

29

In [27]:
to_dummy

Index(['ProductCD', 'card4', 'card6', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_12', 'id_15', 'id_16', 'id_28', 'id_29', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', '_Weekdays', 'P_emaildomain_bin', 'P_emaildomain_suffix', 'R_emaildomain_bin', 'R_emaildomain_suffix', 'had_id', '_Month'], dtype='object')

In [7]:
data_isFraud_1_indexes=list(data[data['isFraud']==1].index)
data_isFraud_0_indexes=list(data[data['isFraud']==0].index)
data_isFraud_1=data.iloc[data_isFraud_1_indexes , : ]
data_isFraud_0=data.iloc[data_isFraud_0_indexes , : ]

target_isFraud_1=data_isFraud_1['isFraud']
print('target_isFraurd_1 shape: ',target_isFraud_1.shape)

features_isFraud_1=data_isFraud_1.drop(['isFraud'],axis=1)
print('features_isFraud_1 shape: ',features_isFraud_1.shape)

target_isFraud_0=data_isFraud_0['isFraud']
print('target_isFraurd_0 shape: ',target_isFraud_0.shape)

features_isFraud_0=data_isFraud_0.drop(['isFraud'],axis=1)
print('features_isFraud_0 shape: ',features_isFraud_0.shape)

target_isFraurd_1 shape:  (6199,)
features_isFraud_1 shape:  (6199, 364)
target_isFraurd_0 shape:  (170964,)
features_isFraud_0 shape:  (170964, 364)


In [0]:
features_isFraud_0_dumm=features_isFraud_0['card4']
features_isFraud_1_dumm=features_isFraud_1['card4']

In [0]:
 features_isFraud_0_dumm=pd.get_dummies(features_isFraud_0_dumm, drop_first=True)

In [0]:
 features_isFraud_1_dumm=pd.get_dummies(features_isFraud_1_dumm, drop_first=True)

In [11]:
data['card4'].unique()

array(['visa', 'mastercard', 'discover', 'american express'], dtype=object)

In [12]:
features_isFraud_1_dumm.head()

,discover,mastercard,visa
170964,0,0,1
170965,0,1,0
170966,0,1,0
170967,0,0,1
170968,0,1,0


In [0]:
def get_kfolds(splits,features,target):
  kf = KFold(n_splits=splits)

  X=features
  Y=target

  X_train_base=[]
  X_test_base=[]
  Y_train_base=[]
  Y_test_base=[]

  for train_index, test_index in kf.split(X):
      #print('train_index', train_index)
      #print('test_index', test_index)
      X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
      Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
      X_train_base.append(X_train)
      X_test_base.append(X_test)
      Y_train_base.append(Y_train)
      Y_test_base.append(Y_test)
  
  return(X_train_base,X_test_base,Y_train_base,Y_test_base)
  

In [14]:
train_features=[]
train_target=[]
test_features=[]
test_target=[]

X_train_0,X_test_0,Y_train_0,Y_test_0=get_kfolds(2,features_isFraud_0_dumm,target_isFraud_0)
X_train_1,X_test_1,Y_train_1,Y_test_1=get_kfolds(2,features_isFraud_1_dumm,target_isFraud_1)
for j in range(len(X_train_0)):
  tr_features = pd.concat([pd.DataFrame(X_train_0[j]), pd.DataFrame(X_train_1[j])], axis=0)
  tr_target = pd.concat([pd.DataFrame(Y_train_0[j]), pd.DataFrame(Y_train_1[j])], axis=0)
  te_features = pd.concat([pd.DataFrame(X_test_0[j]), pd.DataFrame(X_test_1[j])], axis=0)
  te_target = pd.concat([pd.DataFrame(Y_test_0[j]), pd.DataFrame(Y_test_1[j])], axis=0)
  X = np.array(tr_features)
  y = np.array(tr_target)
  sm = SMOTE(random_state=2)
  X_res, y_res = sm.fit_sample(X, y)
  #print("Number transactions X dataset: ", X.shape)
  #print("Number transactions y dataset: ", y.shape)
  #print("Number transactions X_res dataset: ", X_res.shape)
  #print("Number transactions y_res dataset: ", y_res.shape)
  #print("Before OverSampling, counts of label '1': {}".format(sum(y==1)))
  #print("Before OverSampling, counts of label '0': {} \n".format(sum(y==0)))
  #print("After OverSampling, counts of label '1': {}".format(sum(y_res==1)))
  #print("After OverSampling, counts of label '0': {} \n".format(sum(y_res==0)))
  train_features.append(X_res)
  train_target.append(y_res)
  X_te = np.array(te_features)
  y_te = np.array(te_target)

  
  test_features.append(X_te)
  test_target.append(y_te)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:
model = LogisticRegression()  # Choose a model here
for i in range(len(train_features)):
   model.fit(train_features[i], train_target[i] )
   y_pred = model.predict(test_features[i])
   
   print(f'Number fold: {i}')

   f_score=f1_score(test_target[i], y_pred)


   print(f'f-score: {f_score}')



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Number fold: 0
f-score: 0.06813310772701636
Number fold: 1
f-score: 0.06791138628228693


In [0]:
features_isFraud_0_d=features_isFraud_0[['card4','TransactionAmt']]
features_isFraud_1_d=features_isFraud_1[['card4','TransactionAmt']]

In [0]:
col_to_dummy=['card4']

In [0]:
 features_isFraud_0_d=pd.get_dummies(features_isFraud_0_d, columns=col_to_dummy,drop_first=True)

In [0]:
 features_isFraud_1_d=pd.get_dummies(features_isFraud_1_d, columns=col_to_dummy,drop_first=True)

In [20]:
features_isFraud_0_d.head()

,TransactionAmt,card4_discover,card4_mastercard,card4_visa
0,189.95,0,0,1
1,27.95,0,0,1
2,407.50,0,0,1
3,29.00,0,1,0
4,54.50,0,1,0


In [21]:
train_features=[]
train_target=[]
test_features=[]
test_target=[]

X_train_0,X_test_0,Y_train_0,Y_test_0=get_kfolds(2,features_isFraud_0_d,target_isFraud_0)
X_train_1,X_test_1,Y_train_1,Y_test_1=get_kfolds(2,features_isFraud_1_d,target_isFraud_1)
for j in range(len(X_train_0)):
  tr_features = pd.concat([pd.DataFrame(X_train_0[j]), pd.DataFrame(X_train_1[j])], axis=0)
  tr_target = pd.concat([pd.DataFrame(Y_train_0[j]), pd.DataFrame(Y_train_1[j])], axis=0)
  te_features = pd.concat([pd.DataFrame(X_test_0[j]), pd.DataFrame(X_test_1[j])], axis=0)
  te_target = pd.concat([pd.DataFrame(Y_test_0[j]), pd.DataFrame(Y_test_1[j])], axis=0)
  X = np.array(tr_features)
  y = np.array(tr_target)
  sm = SMOTE(random_state=2)
  X_res, y_res = sm.fit_sample(X, y)
  #print("Number transactions X dataset: ", X.shape)
  #print("Number transactions y dataset: ", y.shape)
  #print("Number transactions X_res dataset: ", X_res.shape)
  #print("Number transactions y_res dataset: ", y_res.shape)
  #print("Before OverSampling, counts of label '1': {}".format(sum(y==1)))
  #print("Before OverSampling, counts of label '0': {} \n".format(sum(y==0)))
  #print("After OverSampling, counts of label '1': {}".format(sum(y_res==1)))
  #print("After OverSampling, counts of label '0': {} \n".format(sum(y_res==0)))
  train_features.append(X_res)
  train_target.append(y_res)
  X_te = np.array(te_features)
  y_te = np.array(te_target)

  
  test_features.append(X_te)
  test_target.append(y_te)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [27]:
model = LogisticRegression()  # Choose a model here
for i in range(len(train_features)):
   model.fit(train_features[i], train_target[i] )
   y_pred = model.predict(test_features[i])
   
   print(f'Number fold: {i}')

   f_score=f1_score(test_target[i], y_pred)


   print(f'f-score: {f_score}')



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Number fold: 0
f-score: 0.06961832061068703


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Number fold: 1
f-score: 0.06626434034416825


In [0]:
features_isFraud_0_d=features_isFraud_0[['_Days','card4','TransactionAmt']]
features_isFraud_1_d=features_isFraud_1[['_Days','card4','TransactionAmt']]

In [0]:
col_to_dummy=['_Days','card4']

In [0]:
 features_isFraud_0_d=pd.get_dummies(features_isFraud_0_d, columns=col_to_dummy,drop_first=True)

In [0]:
 features_isFraud_1_d=pd.get_dummies(features_isFraud_1_d, columns=col_to_dummy,drop_first=True)

In [26]:
data['_Days'].unique()

array([15,  2,  8, 31, 17, 10,  1, 29,  5, 18, 25, 13,  7, 30, 20,  6, 14,
       23, 27,  3, 21, 16,  4, 24, 28, 22, 26, 11, 19, 12,  9])

In [27]:
features_isFraud_0_d.head()

,TransactionAmt,_Days_2,_Days_3,_Days_4,_Days_5,_Days_6,_Days_7,_Days_8,_Days_9,_Days_10,_Days_11,_Days_12,_Days_13,_Days_14,_Days_15,_Days_16,_Days_17,_Days_18,_Days_19,_Days_20,_Days_21,_Days_22,_Days_23,_Days_24,_Days_25,_Days_26,_Days_27,_Days_28,_Days_29,_Days_30,_Days_31,card4_discover,card4_mastercard,card4_visa
0,189.95,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,27.95,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,407.50,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,29.00,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,54.50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0


In [0]:
train_features=[]
train_target=[]
test_features=[]
test_target=[]

X_train_0,X_test_0,Y_train_0,Y_test_0=get_kfolds(2,features_isFraud_0_d,target_isFraud_0)
X_train_1,X_test_1,Y_train_1,Y_test_1=get_kfolds(2,features_isFraud_1_d,target_isFraud_1)
for j in range(len(X_train_0)):
  tr_features = pd.concat([pd.DataFrame(X_train_0[j]), pd.DataFrame(X_train_1[j])], axis=0)
  tr_target = pd.concat([pd.DataFrame(Y_train_0[j]), pd.DataFrame(Y_train_1[j])], axis=0)
  te_features = pd.concat([pd.DataFrame(X_test_0[j]), pd.DataFrame(X_test_1[j])], axis=0)
  te_target = pd.concat([pd.DataFrame(Y_test_0[j]), pd.DataFrame(Y_test_1[j])], axis=0)
  X = np.array(tr_features)
  y = np.array(tr_target)
  sm = SMOTE(random_state=2)
  X_res, y_res = sm.fit_sample(X, y)
  #print("Number transactions X dataset: ", X.shape)
  #print("Number transactions y dataset: ", y.shape)
  #print("Number transactions X_res dataset: ", X_res.shape)
  #print("Number transactions y_res dataset: ", y_res.shape)
  #print("Before OverSampling, counts of label '1': {}".format(sum(y==1)))
  #print("Before OverSampling, counts of label '0': {} \n".format(sum(y==0)))
  #print("After OverSampling, counts of label '1': {}".format(sum(y_res==1)))
  #print("After OverSampling, counts of label '0': {} \n".format(sum(y_res==0)))
  train_features.append(X_res)
  train_target.append(y_res)
  X_te = np.array(te_features)
  y_te = np.array(te_target)

  
  test_features.append(X_te)
  test_target.append(y_te)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [0]:
model = LogisticRegression()  # Choose a model here
for i in range(len(train_features)):
   model.fit(train_features[i], train_target[i] )
   y_pred = model.predict(test_features[i])
   
   print(f'Number fold: {i}')

   f_score=f1_score(test_target[i], y_pred)


   print(f'f-score: {f_score}')



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Number fold: 0
f-score: 0.06961832061068703


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Number fold: 1
f-score: 0.06626434034416825


In [0]:
features_isFraud_0_d=features_isFraud_0[['card1','card4','TransactionAmt']]
features_isFraud_1_d=features_isFraud_1[['card1','card4','TransactionAmt']]

In [0]:
col_to_dummy=['card1','card4']

In [0]:
 features_isFraud_0_d=pd.get_dummies(features_isFraud_0_d, columns=col_to_dummy,drop_first=True)

In [0]:
 features_isFraud_1_d=pd.get_dummies(features_isFraud_1_d, columns=col_to_dummy,drop_first=True)

In [0]:
data['card1'].unique()

array([13534,  4663,  8701, ...,  7176, 10586,  4427])

In [0]:
features_isFraud_0_d.head()

,TransactionAmt,card1_1007,card1_1008,card1_1009,card1_1010,card1_1012,card1_1013,card1_1014,card1_1015,card1_1016,card1_1018,card1_1022,card1_1023,card1_1026,card1_1027,card1_1028,card1_1029,card1_1030,card1_1031,card1_1033,card1_1034,card1_1036,card1_1039,card1_1042,card1_1043,card1_1047,card1_1048,card1_1049,card1_1051,card1_1052,card1_1053,card1_1054,card1_1055,card1_1056,card1_1057,card1_1058,card1_1064,card1_1066,card1_1070,card1_1074,card1_1075,card1_1076,card1_1078,card1_1079,card1_1081,card1_1083,card1_1084,card1_1085,card1_1087,card1_1088,card1_1089,card1_1091,card1_1092,card1_1097,card1_1098,card1_1099,card1_1101,card1_1102,card1_1103,card1_1104,card1_1105,card1_1106,card1_1107,card1_1110,card1_1111,card1_1113,card1_1115,card1_1120,card1_1121,card1_1124,card1_1126,card1_1128,card1_1129,card1_1132,card1_1135,card1_1137,card1_1138,card1_1139,card1_1142,card1_1145,card1_1148,card1_1149,card1_1150,card1_1156,card1_1159,card1_1160,card1_1161,card1_1164,card1_1169,card1_1171,card1_1172,card1_1173,card1_1174,card1_1175,card1_1178,card1_1181,card1_1183,card1_1184,card1_1186,card1_1193,card1_1195,card1_1197,card1_1198,card1_1200,card1_1201,card1_1206,card1_1208,card1_1210,card1_1211,card1_1213,card1_1214,card1_1215,card1_1216,card1_1219,card1_1220,card1_1222,card1_1223,card1_1225,card1_1229,card1_1230,card1_1231,card1_1232,card1_1233,card1_1234,card1_1238,card1_1240,card1_1242,card1_1243,card1_1244,card1_1245,card1_1246,card1_1247,card1_1252,card1_1253,card1_1255,card1_1257,card1_1258,card1_1260,card1_1262,card1_1268,card1_1269,card1_1271,card1_1273,card1_1274,card1_1275,card1_1277,card1_1279,card1_1281,card1_1282,card1_1284,card1_1285,card1_1286,card1_1288,card1_1290,card1_1292,card1_1296,card1_1297,card1_1299,card1_1300,card1_1301,card1_1304,card1_1306,card1_1308,card1_1310,card1_1311,card1_1314,card1_1317,card1_1322,card1_1323,card1_1324,card1_1327,card1_1329,card1_1331,card1_1332,card1_1333,card1_1334,card1_1335,card1_1336,card1_1337,card1_1338,card1_1339,card1_1341,card1_1342,card1_1344,card1_1346,card1_1355,card1_1357,card1_1358,card1_1359,card1_1360,card1_1361,card1_1363,card1_1364,card1_1365,card1_1367,card1_1368,card1_1372,card1_1373,card1_1375,card1_1377,card1_1381,card1_1386,card1_1387,card1_1388,card1_1392,card1_1393,card1_1395,card1_1398,card1_1399,card1_1404,card1_1406,card1_1407,card1_1408,card1_1409,card1_1410,card1_1411,card1_1412,card1_1416,card1_1417,card1_1419,card1_1422,card1_1423,card1_1424,card1_1426,card1_1427,card1_1428,card1_1429,card1_1431,card1_1433,card1_1435,card1_1440,card1_1441,card1_1443,card1_1444,card1_1449,card1_1450,card1_1453,card1_1454,card1_1458,card1_1460,card1_1463,card1_1465,card1_1468,card1_1469,card1_1470,card1_1471,card1_1473,card1_1474,card1_1475,card1_1476,...,card1_17939,card1_17940,card1_17941,card1_17942,card1_17943,card1_17945,card1_17946,card1_17947,card1_17949,card1_17953,card1_17955,card1_17957,card1_17958,card1_17960,card1_17962,card1_17965,card1_17966,card1_17967,card1_17968,card1_17969,card1_17970,card1_17971,card1_17975,card1_17976,card1_17978,card1_17983,card1_17984,card1_17985,card1_17995,card1_17996,card1_17998,card1_17999,card1_18001,card1_18005,card1_18011,card1_18012,card1_18013,card1_18014,card1_18015,card1_18018,card1_18020,card1_18026,card1_18027,card1_18028,card1_18030,card1_18031,card1_18034,card1_18035,card1_18036,card1_18037,card1_18039,card1_18042,card1_18043,card1_18045,card1_18046,card1_18050,card1_18052,card1_18054,card1_18055,card1_18062,card1_18063,card1_18068,card1_18072,card1_18073,card1_18080,card1_18081,card1_18084,card1_18086,card1_18087,card1_18088,card1_18089,card1_18092,card1_18093,card1_18094,card1_18095,card1_18096,card1_18098,card1_18099,card1_18100,card1_18101,card1_18103,card1_18104,card1_18105,card1_18106,card1_18107,card1_18109,card1_18110,card1_18111,card1_18112,card1_18117,card1_18118,card1_18122,card1_18123,card1_18124,card1_18127,card1_18129,card1_18130,card1_18132,card1_18135,card1_18136,card1_18137,card1_18141,card1_18144,card1

In [0]:
train_features=[]
train_target=[]
test_features=[]
test_target=[]

X_train_0,X_test_0,Y_train_0,Y_test_0=get_kfolds(2,features_isFraud_0_d,target_isFraud_0)
X_train_1,X_test_1,Y_train_1,Y_test_1=get_kfolds(2,features_isFraud_1_d,target_isFraud_1)
for j in range(len(X_train_0)):
  tr_features = pd.concat([pd.DataFrame(X_train_0[j]), pd.DataFrame(X_train_1[j])], axis=0)
  tr_target = pd.concat([pd.DataFrame(Y_train_0[j]), pd.DataFrame(Y_train_1[j])], axis=0)
  te_features = pd.concat([pd.DataFrame(X_test_0[j]), pd.DataFrame(X_test_1[j])], axis=0)
  te_target = pd.concat([pd.DataFrame(Y_test_0[j]), pd.DataFrame(Y_test_1[j])], axis=0)
  X = np.array(tr_features)
  y = np.array(tr_target)
  sm = SMOTE(random_state=2)
  X_res, y_res = sm.fit_sample(X, y)
  #print("Number transactions X dataset: ", X.shape)
  #print("Number transactions y dataset: ", y.shape)
  #print("Number transactions X_res dataset: ", X_res.shape)
  #print("Number transactions y_res dataset: ", y_res.shape)
  #print("Before OverSampling, counts of label '1': {}".format(sum(y==1)))
  #print("Before OverSampling, counts of label '0': {} \n".format(sum(y==0)))
  #print("After OverSampling, counts of label '1': {}".format(sum(y_res==1)))
  #print("After OverSampling, counts of label '0': {} \n".format(sum(y_res==0)))
  train_features.append(X_res)
  train_target.append(y_res)
  X_te = np.array(te_features)
  y_te = np.array(te_target)

  
  test_features.append(X_te)
  test_target.append(y_te)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [0]:
model = LogisticRegression()  # Choose a model here
for i in range(len(train_features)):
   model.fit(train_features[i], train_target[i] )
   y_pred = model.predict(test_features[i])
   
   print(f'Number fold: {i}')

   f_score=f1_score(test_target[i], y_pred)


   print(f'f-score: {f_score}')



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Number fold: 0
f-score: 0.06961832061068703


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Number fold: 1
f-score: 0.06626434034416825
